## Audio Monitor

Monitors the audio by continuously recording and plotting the recorded audio.

You will need to start osp running in a terminal.

In [1]:
# make Jupyter use the whole width of the browser
from IPython.display import Image, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Pull in all the necessary python modules
import plotly.graph_objects as go
from IPython.display import Audio
from osp_control import OspControl
import os
import json
import numpy as np
import time
import ipywidgets as widgets


In [3]:
# Connect to OSP process
osp = OspControl()  # connects to a local process

# set the number of bands and mute mic
osp.send({"method": "set", "data": {"num_bands": 11}})
osp.send_chan({"alpha": 0})

{'Result': 'success'}

In [4]:
def plot_fft(fig, title, out1, update=True):
    ftrans = np.abs(np.fft.fft(out1)/len(out1))
    outlen = len(out1)
    values = np.arange(int(outlen/2))
    period = outlen/32000
    frequencies = values/period

    if update:
        with fig.batch_update():
            fig.data[0].y = ftrans
            fig.data[0].x = frequencies
            fig.update_layout()
            return
        
   
    fig.add_trace(go.Scatter(x=frequencies, line_color='green', opacity=1, y=ftrans))

    fig.update_layout(title=title,
                    xaxis_title='Frequency',
                    yaxis_title='Amplitude',
                    template='plotly_white')

In [5]:
def plot_mono(fig, title, out1, lab1, rate=48000, update=True):

    # Create x values.
    x = np.array(range(len(out1)))/rate
    if update:
        with fig.batch_update():
            fig.data[0].y = out1
            fig.data[0].x = x
            fig.update_layout()
            return
    fig.add_trace(go.Scatter(x=x,
                             y=out1,
                             name=lab1,
                             opacity=1))
    fig.update_layout(title=title,
#                       yaxis_range=[-1,1],
                      xaxis_title='Time(sec)',
                      yaxis_title='Amplitude',
                      template='plotly_white')

In [6]:
def monitor(afig, ffig, interval=2):
    update=False
    while True:
        # set record file name
        filename = os.path.join(os.getcwd(), 'tmpaudio')
        osp.send_chan({'audio_rfile': filename})

        #start recording output
        osp.send_chan({"audio_record": 1})

        # record for {interval} seconds
        time.sleep(interval)

        #stop recording
        osp.send_chan({"audio_record": 0})

        # read the saved recording data
        with open(filename, "rb") as f:
            data = f.read()
        data = np.frombuffer(data, dtype=np.float32)
        data = np.reshape(data, (2, -1), 'F')
        
        # plot
        plot_mono(afig, '', data[0], 'out', update=update)
        plot_fft(ffig, '',  data[0], update=update)
        if update == False:
            update = True

In [7]:
audio_fig = go.FigureWidget()
fft_fig = go.FigureWidget()
widgets.VBox([audio_fig, fft_fig])

    'data': [], 'layout': {'template': '...'}
}), FigureWidget({
    'data': [],…

In [8]:
monitor(audio_fig, fft_fig)

OSPNotRunningException: Could not connect to the osp process on the given hostname/port